## Applying the bias scan tool on a loan approval classifier
In this notebook, the bias scan tool is applied on a XGBoost loan approval classifier. The bias scan tool is based on an implementation of the k-means Hierarchical Bias Aware Clustering (HBAC) method\*. The python script `./helper_functions.py` contains functions that execute the bias scan. A conceptual description how the bias scan works, including the rationale why k-means is chosen as a clustering algorithm and paramater choices, can be found in the [bias scan report](https://github.com/NGO-Algorithm-Audit/Bias_scan/blob/master/Bias_scan_tool_report.pdf) (technical documentation).

An XGBoost classifier is trained to make predictions on the German Credit\*\* data set. Details on pre-processing steps performed on this dataset and the trained XGBoost classifier are provided in the `../case_studies/Loan_approval_classifier/GermanCredit_classifier.ipynb` notebook.

\* Misztal-Radecka, Indurkya, *Information Processing and Management*. Bias-Aware Hierarchical Clustering for detecting the discriminated groups of users in recommendation systems (2021).

\*\* the original dataset, in the form provided by Prof. Hofmann, contains categorical/symbolic attributes which is contained in the file "german.data" [[link to dataset]](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29)

### Overview of the notebook
1. Load data and pre-processing
2. Bias scan using k-means clustering
3. Clustering results
4. Statistical testing of inter-cluster difference 

### Load libraries

In [3]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Markdown, display

# helper functions
from helper_functions import *

# sklearn
from sklearn.preprocessing import StandardScaler

# welch's t-test
import scipy.stats as stats

### 1. Load data and pre-processing
#### Load data

In [9]:
path = '../classifiers/Loan_approval_classifier/pred_XGBoost.csv'

# read data
entire_dataset = pd.read_csv(path)
entire_dataset.head()

,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,status=A11,status=A12,...,skill_level=A172,skill_level=A173,skill_level=A174,telephone=A191,telephone=A192,foreign_worker=A201,foreign_worker=A202,predicted_class,true_class,errors
0,21.0,2993.0,3.0,2.0,28.0,2.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0,0.0,0.0
1,30.0,3656.0,4.0,4.0,49.0,2.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0,0.0,0.0
2,12.0,1255.0,4.0,4.0,61.0,2.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0,0.0,0.0
3,8.0,1414.0,4.0,2.0,33.0,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0,0.0,0.0
4,12.0,691.0,4.0,3.0,35.0,2.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0,1.0,1.0


#### Data cleaning

In [10]:
features = entire_dataset.drop(['predicted_class', 'true_class', 'errors'], axis=1)
features.head()

,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,status=A11,status=A12,...,housing=A152,housing=A153,skill_level=A171,skill_level=A172,skill_level=A173,skill_level=A174,telephone=A191,telephone=A192,foreign_worker=A201,foreign_worker=A202
0,21.0,2993.0,3.0,2.0,28.0,2.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,30.0,3656.0,4.0,4.0,49.0,2.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,12.0,1255.0,4.0,4.0,61.0,2.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,8.0,1414.0,4.0,2.0,33.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,12.0,691.0,4.0,3.0,35.0,2.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


#### Scaling data

In [11]:
full_data = init_GermanCredit_dataset(entire_dataset,features)
full_data.head()

KeyError: "['FP_errors', 'FN_errors'] not found in axis"

### 2. Bias scan using k-means clustering
Clustering algorithms parameters:

In [5]:
clustering_paramaters = {
    "n_clusters": 3,
    "init": "k-means++",
    "n_init": 20,
    "max_iter": 300,
    "random_state": 10,
}

Specify:
- Minimal splittable cluster size
- Minimal acceptable cluster size

In [6]:
# minimal splittable cluster size
split_cluster_size = round(0.05 * len(full_data))
print("minimal splittable cluster size: ", split_cluster_size)

# minimal acceptable cluster size
acc_cluster_size = round(0.03 * len(full_data))
print("minimal splitacceptabletable cluster size: ", acc_cluster_size)

minimal splittable cluster size:  15
minimal splitacceptabletable cluster size:  9


Performing bias scan using helper functions.

In [7]:
metric = 'accuracy'
iterations_max = 20
x = 0 # initial cluster number
initial_bias = 0
variance_list = []
average_accuracy = accuracy(full_data, metric)
minimal_splittable_cluster_size = 18
minimal_acceptable_cluster_size = 10
print("average_accuracy is: ", average_accuracy) 

for i in range(1, iterations_max):
    if i != 1:
        variance_list.append(calculate_variance(full_data,metric)) 
    full_data['new_clusters'] = -1
    candidate_cluster = full_data.loc[full_data['clusters'] == x] 

    if len(candidate_cluster) < minimal_splittable_cluster_size:
        x = get_random_cluster(full_data['clusters'])
        continue
    
    # Apply Kmeans 
    kmeans_algo = KMeans(**clustering_paramaters).fit(candidate_cluster.drop(['clusters', 'new_clusters', 'predicted_class', 'true_class', 'errors'], axis=1))
    
    candidate_cluster['new_clusters'] = pd.DataFrame(kmeans_algo.predict(candidate_cluster.drop(['clusters', 'new_clusters', 'predicted_class', 'true_class', 'errors'], axis=1)),index=candidate_cluster.index) 
    full_data['new_clusters'] = candidate_cluster['new_clusters'].combine_first(full_data['new_clusters'])

    max_abs_bias = get_max_negative_bias(full_data,metric) # was get_max_abs_bias, but now it only finds the discriminated clusters
    min_new_size = get_min_cluster_size(full_data)
    
    if (max_abs_bias <= initial_bias) & (min_new_size > minimal_acceptable_cluster_size): #abs: >
        # add new cluster
        n_cluster = max(full_data['clusters'])
        full_data['clusters'][full_data['new_clusters'] == 1] =  n_cluster + 1
        
        x = get_next_cluster(full_data)
        initial_bias = max_abs_bias
    else:
        x = get_random_cluster(full_data['clusters'])
        
print('done')

average_accuracy is:  0.7333333333333333
done


### 3. Analysing clustering results
Identifying cluster with most negative bias.

In [8]:
c = get_max_bias_cluster(full_data, metric)
max_bias = round(bias_acc(full_data, metric, c, "clusters"), 2)
highest_biased_cluster = full_data[full_data['clusters']==c]
print(f"cluster {c} has the highest negative bias: " + str(max_bias))
print("#elements in highest biased cluster:", len(highest_biased_cluster))

1 has bias 0.018478895156584296
0 has bias 0.06391388445042467
2 has bias -0.04983388704318936
4 has bias -0.02188782489740082
3 has bias -0.03571428571428581
cluster 2 has the highest negative bias: -0.05
#elements in highest biased cluster: 42


Accuracy of classifier on full dataset and on cluster with highest discrimination bias.

In [9]:
accuracy_full_data = accuracy(full_data, metric)
most_biased_cluster_kmeans_aware = bias_acc(full_data, metric, c, 'clusters')
full_data[full_data['clusters']==c]

print('General accuracy of classifier on this dataset:', accuracy_full_data)

General accuracy of classifier on this dataset: 0.7333333333333333


Select and print discriminated cluser.

In [10]:
c=c
discriminated_cluster = full_data[full_data['clusters']==c]
print('Number of instances in discriminated cluster:', len(discriminated_cluster))
print('Number of errors in discriminated clusters: ', len(discriminated_cluster.loc[discriminated_cluster['errors']==1]))
discriminated_cluster

Number of instances in discriminated cluster: 42
Number of errors in discriminated clusters:  13


,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,status=A11,status=A12,...,skill_level=A174,telephone=A191,telephone=A192,foreign_worker=A201,foreign_worker=A202,predicted_class,true_class,errors,clusters,new_clusters
5,0.314895,-0.667366,0.877513,1.084768,1.884043,0.943341,-0.447214,1.323742,1.603567,-0.592749,...,2.209605,-1.191367,1.191367,0.195096,-0.195096,0,0.0,0.0,2,2.0
8,2.350125,0.501838,-0.049437,1.084768,-1.016739,-0.711644,-0.447214,1.323742,1.603567,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,1,1.0,0.0,2,0.0
15,-0.193912,2.083702,-1.903339,-0.764268,-1.104641,0.943341,-0.447214,1.323742,1.603567,-0.592749,...,-0.452570,-1.191367,1.191367,0.195096,-0.195096,1,0.0,1.0,2,1.0
22,-0.702720,-0.420378,-0.976388,-0.764268,0.741311,-0.711644,-0.447214,1.323742,-0.623610,-0.592749,...,-0.452570,-1.191367,1.191367,0.195096,-0.195096,0,0.0,0.0,2,1.0
23,0.314895,0.012802,-0.976388,-0.764268,-1.016739,-0.711644,-0.447214,-0.755434,1.603567,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,1,0.0,1.0,2,1.0
27,-0.957124,-0.616068,-0.049437,1.084768,-1.192544,-0.711644,-0.447214,1.323742,1.603567,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,0,1.0,1.0,2,0.0
41,-0.448316,-0.459516,-0.976388,-1.688786,-0.840934,0.943341,-0.447214,1.323742,-0.623610,1.687055,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,0,1.0,1.0,2,0.0
44,-0.957124,-0.662426,0.877513,1.084768,0.477604,0.943341,-0.447214,1.323742,-0.623610,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,0,0.0,0.0,2,1.0
50,-0.702720,-0.979711,-0.049437,0.160250,0.565506,0.943341,-0.447214,1.323742,-0.623610,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,0,0.0,0.0,2,1.0
56,-0.193912,0.048140,-1.903339,1.084768,-0.840934,-0.711644,-0.447214,1.323742,1.603567,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,1,1.0,0.0,2,0.0


### 4. Statistical testing of inter-cluster difference 
Compute difference between cluster with highest discrimination bias and rest of dataset.

In [11]:
not_discriminated = full_data[full_data['clusters']!=c]
difference = (discriminated_cluster.mean()) - (not_discriminated.mean()) 
diff_dict = difference.to_dict()
difference

month                             -0.448517
credit_amount                     -0.259854
investment_as_income_percentage   -0.622073
residence_since                    0.493486
age                               -0.466769
                                     ...   
predicted_class                    0.084718
true_class                         0.122370
errors                             0.049834
clusters                           0.038760
new_clusters                       2.142857
Length: 63, dtype: float64

Unscaling the data. Print summary of cluster with highest discrimination bias and the rest of data.

In [12]:
# unscaling the discriminated cluster
unscaled_discriminated = full_data.loc[discriminated_cluster.index, :]

# unscaled other data
unscaled_remaining = full_data.drop(discriminated_cluster.index)

display(unscaled_remaining.describe())
display(unscaled_discriminated.describe())

print(discriminated_cluster.index)

,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,status=A11,status=A12,...,skill_level=A174,telephone=A191,telephone=A192,foreign_worker=A201,foreign_worker=A202,predicted_class,true_class,errors,clusters,new_clusters
count,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,...,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.000000,258.0
mean,0.062792,0.036380,0.087090,-0.069088,0.065348,-0.018859,0.010400,-0.078493,-0.036602,-0.000707,...,0.042719,-0.002834,0.002834,-0.031760,0.031760,0.224806,0.306202,0.259690,1.961240,-1.0
std,1.026742,1.047365,0.961306,1.002401,1.005166,0.972772,1.011167,0.976178,0.983127,1.001556,...,1.037983,1.002439,1.002439,1.077071,1.077071,0.418266,0.461810,0.439317,1.594754,0.0
min,-1.381130,-1.006689,-1.903339,-1.688786,-1.368349,-0.711644,-0.447214,-0.755434,-0.623610,-0.592749,...,-0.452570,-1.191367,-0.839372,-5.125693,-0.195096,0.000000,0.000000,0.000000,0.000000,-1.0
25%,-0.702720,-0.635922,-0.976388,-0.764268,-0.665129,-0.711644,-0.447214,-0.755434,-0.623610,-0.592749,...,-0.452570,-1.191367,-0.839372,0.195096,-0.195096,0.000000,0.000000,0.000000,1.000000,-1.0
50%,-0.193912,-0.333742,0.877513,0.160250,-0.137714,-0.711644,-0.447214,-0.755434,-0.623610,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,0.000000,0.000000,0.000000,1.000000,-1.0
75%,0.314895,0.324672,0.877513,1.084768,0.719335,0.943341,-0.447214,1.323742,1.603567,1.687055,...,-0.452570,0.839372,1.191367,0.195096,-0.195096,0.000000,1.000000,1.000000,4.000000,-1.0
max,3.367741,4.481766,0.877513,1.084768,3.466288,4.253312,2.236068,1.323742,1.603567,1.687055,...,2.209605,0.839372,1.191367,0.195096,5.125693,1.000000,1.000000,1.000000,4.000000,-1.0


,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,status=A11,status=A12,...,skill_level=A174,telephone=A191,telephone=A192,foreign_worker=A201,foreign_worker=A202,predicted_class,true_class,errors,clusters,new_clusters
count,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,...,42.000000,42.000000,42.000000,42.000000,4.200000e+01,42.000000,42.000000,42.000000,42.0,42.000000
mean,-0.385725,-0.223475,-0.534983,0.424398,-0.401421,0.115849,-0.063888,0.482171,0.224839,0.004342,...,-0.262414,0.017406,-0.017406,0.195096,-1.950956e-01,0.309524,0.428571,0.309524,2.0,1.142857
std,0.729262,0.617604,1.087921,0.896719,0.889892,1.170251,0.950335,1.032926,1.094674,1.014514,...,0.693926,1.008862,1.008862,0.000000,8.427606e-17,0.467901,0.500870,0.467901,0.0,0.718108
min,-1.381130,-0.979711,-1.903339,-1.688786,-1.368349,-0.711644,-0.447214,-0.755434,-0.623610,-0.592749,...,-0.452570,-1.191367,-0.839372,0.195096,-1.950956e-01,0.000000,0.000000,0.000000,2.0,0.000000
25%,-0.702720,-0.640007,-1.671601,-0.533139,-1.082666,-0.711644,-0.447214,-0.755434,-0.623610,-0.592749,...,-0.452570,-1.191367,-0.839372,0.195096,-1.950956e-01,0.000000,0.000000,0.000000,2.0,1.000000
50%,-0.702720,-0.392069,-0.976388,1.084768,-0.753031,-0.711644,-0.447214,1.323742,-0.623610,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-1.950956e-01,0.000000,0.000000,0.000000,2.0,1.000000
75%,-0.193912,0.007862,0.877513,1.084768,0.323774,0.943341,-0.447214,1.323742,1.603567,1.117104,...,-0.452570,0.839372,1.191367,0.195096,-1.950956e-01,1.000000,1.000000,1.000000,2.0,2.000000
max,2.350125,2.083702,0.877513,1.084768,1.884043,4.253312,2.236068,1.323742,1.603567,1.687055,...,2.209605,0.839372,1.191367,0.195096,-1.950956e-01,1.000000,1.000000,1.000000,2.0,2.000000


Int64Index([  5,   8,  15,  22,  23,  27,  41,  44,  50,  56,  60,  72,  73,
             81,  82,  88, 103, 114, 115, 118, 119, 121, 124, 134, 147, 151,
            157, 174, 176, 188, 190, 197, 213, 219, 231, 236, 237, 247, 249,
            262, 274, 285],
           dtype='int64')


In [13]:
unscaled_discriminated

,month,credit_amount,investment_as_income_percentage,residence_since,age,number_of_credits,people_liable_for,sex,status=A11,status=A12,...,skill_level=A174,telephone=A191,telephone=A192,foreign_worker=A201,foreign_worker=A202,predicted_class,true_class,errors,clusters,new_clusters
5,0.314895,-0.667366,0.877513,1.084768,1.884043,0.943341,-0.447214,1.323742,1.603567,-0.592749,...,2.209605,-1.191367,1.191367,0.195096,-0.195096,0,0.0,0.0,2,2.0
8,2.350125,0.501838,-0.049437,1.084768,-1.016739,-0.711644,-0.447214,1.323742,1.603567,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,1,1.0,0.0,2,0.0
15,-0.193912,2.083702,-1.903339,-0.764268,-1.104641,0.943341,-0.447214,1.323742,1.603567,-0.592749,...,-0.452570,-1.191367,1.191367,0.195096,-0.195096,1,0.0,1.0,2,1.0
22,-0.702720,-0.420378,-0.976388,-0.764268,0.741311,-0.711644,-0.447214,1.323742,-0.623610,-0.592749,...,-0.452570,-1.191367,1.191367,0.195096,-0.195096,0,0.0,0.0,2,1.0
23,0.314895,0.012802,-0.976388,-0.764268,-1.016739,-0.711644,-0.447214,-0.755434,1.603567,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,1,0.0,1.0,2,1.0
27,-0.957124,-0.616068,-0.049437,1.084768,-1.192544,-0.711644,-0.447214,1.323742,1.603567,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,0,1.0,1.0,2,0.0
41,-0.448316,-0.459516,-0.976388,-1.688786,-0.840934,0.943341,-0.447214,1.323742,-0.623610,1.687055,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,0,1.0,1.0,2,0.0
44,-0.957124,-0.662426,0.877513,1.084768,0.477604,0.943341,-0.447214,1.323742,-0.623610,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,0,0.0,0.0,2,1.0
50,-0.702720,-0.979711,-0.049437,0.160250,0.565506,0.943341,-0.447214,1.323742,-0.623610,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,0,0.0,0.0,2,1.0
56,-0.193912,0.048140,-1.903339,1.084768,-0.840934,-0.711644,-0.447214,1.323742,1.603567,-0.592749,...,-0.452570,0.839372,-0.839372,0.195096,-0.195096,1,1.0,0.0,2,0.0


####  Test to check statistical significance of inter-cluster difference (per feature)
Applying a Welch’s two-samples t-test for unequal variances to examine whether the differences in means for each feature are statistically significant and store results in a dictionary.

In [14]:
welch_dict = {}

features = [col for col in full_data.columns.tolist() if col not in ['scaled_errors','predicted_class','true_class','errors','clusters','new_clusters']]
    
for i in features:
    welch_i = stats.ttest_ind(unscaled_discriminated[i], unscaled_remaining[i], equal_var=False)
    
    # attach to dictionary
    welch_dict[i] = welch_i.pvalue
    
welch_dict

{'month': 0.0009040491339193922,
 'credit_amount': 0.026994778660318502,
 'investment_as_income_percentage': 0.0009921452505663717,
 'residence_since': 0.0019020609376420013,
 'age': 0.00301601517916447,
 'number_of_credits': 0.4826339811389547,
 'people_liable_for': 0.6433287303514661,
 'sex': 0.0017912125248196748,
 'status=A11': 0.15158660315804853,
 'status=A12': 0.9762030093392534,
 'status=A13': 0.42461384641931765,
 'status=A14': 0.25734497072397244,
 'credit_history=A30': 0.1538145529341457,
 'credit_history=A31': 0.2749496430927019,
 'credit_history=A32': 0.4976222676432118,
 'credit_history=A33': 0.1994039544607175,
 'credit_history=A34': 0.5370054068234992,
 'purpose=A40': 0.5689556647646783,
 'purpose=A41': 0.8203612176847211,
 'purpose=A410': 0.08326217321653268,
 'purpose=A42': 0.04020730116525032,
 'purpose=A43': 0.5890851021973789,
 'purpose=A44': 0.02505971208819176,
 'purpose=A45': 0.004476966607310399,
 'purpose=A46': 0.622448304942083,
 'purpose=A48': 0.861768786688

#### p-values
A small p-value (p < 0.05) indicates that it is unlikely to observe inter-cluster difference due to chance. Sort difference on statistical significance (p-value).

In [16]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option("display.max_rows", None, "display.max_columns", None)
cluster_analysis_df = pd.DataFrame([diff_dict, welch_dict]).T
cluster_analysis_df.columns = ['difference','p-value']
cluster_analysis_df = cluster_analysis_df.sort_values('p-value')
cluster_analysis_df

,difference,p-value
housing=A151,2.45416,0.00000
housing=A152,-1.81941,0.00000
property=A124,-0.46915,0.00000
housing=A153,-0.40180,0.00000
other_debtors=A101,0.34289,0.00000
employment=A71,-0.31077,0.00001
installment_plans=A142,-0.29377,0.00002
other_debtors=A103,-0.28499,0.00003
foreign_worker=A202,-0.22686,0.00083
foreign_worker=A201,0.22686,0.00083


#### Conclusion
Suspected negative bias by the classifier on the basis of (p<0.05):
- Housing (`housing=A151,A152,A153`);
- Property real estate or unknown (`property=A121,A124`);
- Other debtors (`other_debtors=A101,A102,A103`);
- Employment (`employment=A71`);
- Installment plans (`installment_plans=A142,A=143`);
- Foreign worker (`foreign_worker=A201,A202`);
- Month (`month`);
- Investment as percentage of annual income (`investment_as_income_percentage`);
- Gender (`sex`);
- Present residence since (`residence since`);
- Age in years (`age`)
- Type of property (`property=A42,A44,A45`);
- Job type (`skill_level=A174`);
- Amount of credit requested (`credit_amount`).

This means that loan applicants with the above features on average are significantly less often approved by the XGBoost classifier. An exact description of the features can be found [here](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29).

#### What's next?
Qualitative assessment with the help of subject matter experts to verify the measured quantitaive disparities. Additionally, sensitivity testing would be beneficial to shed light into the robustness of the bias scan tool. 